# Surf's Up

### Imports, Base instantiation

In [45]:
import os
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

import pymysql
pymysql.install_as_MySQLdb()

Base = declarative_base()

### Read in CSVs

In [46]:
csv1 = pd.read_csv('../SurfsUp/Output/stations_clean.csv')
csv2 = pd.read_csv('../SurfsUp/Output/measurements_clean.csv')
stations = pd.DataFrame(csv1)
measurements = pd.DataFrame(csv2)

### Create hawaii.sqlite database

In [47]:
engine = create_engine('sqlite:///hawaii.sqlite')

### Set up ORM templates

In [48]:
class Stations(Base):
    __tablename__ = 'hawaii_stations'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
class Measurement(Base):
    __tablename__ = 'hawaii_measurements'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)
    

### Initiate mapping with create_all

In [49]:
conn = engine.connect()
Base.metadata.create_all(conn)

### Load csv into database

In [50]:
station_dict = csv1.to_dict(orient='records')
measurement_dict = csv2.to_dict(orient='records')

In [51]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [52]:
station_dict[0]

{'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [53]:
measurement_dict[0]

{'date': '2010-01-01', 'prcp': 0.08, 'station': 'USC00519397', 'tobs': 65}

In [54]:
table_s = sqlalchemy.Table('hawaii_stations', metadata, autoload=True)
table_m = sqlalchemy.Table('hawaii_measurements', metadata, autoload=True)

In [57]:
conn.execute(table_s.delete())
conn.execute(table_m.delete())
conn.execute(table_s.insert(), station_dict)
conn.execute(table_m.insert(), measurement_dict)

In [58]:
conn.execute('SELECT * FROM hawaii_stations LIMIT 5').fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

In [59]:
conn.execute('SELECT * FROM hawaii_measurements LIMIT 5').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70)]